In [1]:
import torch
import torch.nn as nn

import unidecode
import string
import random
import re
import time, math

In [2]:
num_epochs = 2000 # 에포크
print_every = 100 # 출력 사이클
plot_every = 10   # 플롯 사이클
chunk_len = 200   # 청크 길이 
hidden_size = 100 #
batch_size = 1    # 배치크기
num_layers = 1    # 레이어 수 
embedding_size = 70 # 임베딩 벡터 크기
lr = 0.002        # 학습률

In [6]:
# import 했던 string에서 출력가능한 문자들을 다 불러옵니다. 
all_characters = string.printable

# 출력가능한 문자들의 개수를 저장해놓습니다.
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [3]:
file = unidecode.unidecode(open('./data/input.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 1115393


In [4]:
# 랜덤 청크 ( 길이: chunk_len) - 텍스트의 랜덤한 부분 로딩
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

print(random_chunk())


labour and effect one thing specially.

GREMIO:
What's that, I pray?

HORTENSIO:
Marry, sir, to get a husband for her sister.

GREMIO:
A husband! a devil.

HORTENSIO:
I say, a husband.

GREMIO:
I say,


In [7]:
# 데이터 to tensor (index)
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [8]:
# 랜덤한 파트의 데이터 생성
# Eg. Hello => 입력: Hell, 타겟: ello
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [21]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, mode):
        super(RNN, self).__init__()
        self.input_size = 1
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.mode = mode
        
        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        if mode == 'basic':
            self.rnn = nn.RNN(self.embedding_size,self.hidden_size,self.num_layers)
        elif mode == 'LSTM':
            self.rnn = nn.LSTM(self.embedding_size,self.hidden_size,self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)
        
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        return out,hidden

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        if self.mode == 'basic':
            return hidden
        if self.mode == 'LSTM':
            cell = torch.zeros(self.num_layers,batch_size,self.hidden_size)
            return hidden, cell
        

In [22]:
rnn = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers,'basic')
rnn_lstm = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers,'LSTM')

In [23]:
rnn_optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
rnn_lstm_optimizer = torch.optim.Adam(rnn_lstm.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [30]:
# 임의의 문자(start_str)로 시작하는 길이 200짜리 모방 글을 생성하는 코드입니다.
def test(model):
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    print("Start string: ", start_str,end="")
    print("\nPredicted string: ")
    
    for i in range(200):
        output,hidden = model(x,hidden)

        # 여기서 max값을 사용하지 않고 multinomial을 사용하는 이유는 만약 max 값만 쓰는 경우에
        # 생성되는 텍스트가 다 the the the the the 이런식으로 나오기 때문입니다.
        # multinomial 함수를 통해 높은 값을 가지는 문자들중에 램덤하게 다음 글자를 뽑아내는 방식으로 자연스러운 텍스트를 생성해냅니다.
        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [ ]:
for i in range(num_epochs):
    # 랜덤한 텍스트 덩어리를 샘플링하고 이를 인덱스 텐서로 변환합니다. 
    inp,label = random_training_set()
    hidden = rnn.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    rnn_optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden = rnn(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    rnn_optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test(rnn)
        print("\n","="*100)


 tensor([2.1642], grad_fn=<DivBackward0>) 

Start string:  b
Predicted string: 
est,
And my liffing and murb,
And what of tha sthe fi that for speane of she my over star whom bear of be lave his dear;
sward, is you wor and ied my phander harper:
Ang, at litten th
And amdem ther b

 tensor([1.8505], grad_fn=<DivBackward0>) 

Start string:  b
Predicted string: 
lere me were hear in nod ovill thee this gorey.

CORANICE:
Thet luctiens.

Toree I goldull grourd muct and deartanes lotter ap art fell, I condersher Vouces;
Have gous this for monsty ho siil not grou

 tensor([2.0188], grad_fn=<DivBackward0>) 

Start string:  b
Predicted string: 
est,
Shy enters on will senteant, lord our with me to some but matter; le jood the deast be seantyen o mord frother durnes, of ik the I yet me, is wordeant sownobles,
Beacomafest so the lead the kepre

 tensor([1.9188], grad_fn=<DivBackward0>) 

Start string:  b
Predicted string: 
e.

DUKE CCARDY OF EONE:
Kess,
And when hem
roud, me dear weri:
Seen be a

In [ ]:
for i in range(num_epochs):
    # 랜덤한 텍스트 덩어리를 샘플링하고 이를 인덱스 텐서로 변환합니다. 
    inp,label = random_training_set()
    hidden = rnn_lstm.init_hidden()

    loss = torch.tensor([0]).type(torch.FloatTensor)
    rnn_lstm_optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y,hidden = rnn_lstm(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    rnn_lstm_optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test(rnn_lstm)
        print("\n","="*100)